In [1]:
import pandas as pd
adrian = pd.read_csv('label_text_Adrian.csv', header = None)
adrian

,0
0,Boneless Lamb Product of Australia L Leg-Chump...
1,Keep Frozen Product of Australia Beef Burger B...
2,Premium Cooked Frozen Octopus Tentacles Sous V...
3,Frozen Ocean Trout Fillet Skin On Individually...
4,Gourmet Beef Burger Patties Part Cooked 120g T...
5,Tassal Tasmanian Salmon Product of Australia T...
6,SL SFRC Date 4/07/24 Packed Snapper FLTS Red C...
7,Frozen Thigh Fillet N/B-Diced 94234244564 Prod...
8,(30PCS/BX) Frozen Refrigeration: Keep Frozen(S...
9,SEFSSFFF Packed Date 25/06/24 Snapper FLTS Sad...


In [2]:
import pandas as pd
warra = pd.read_csv('label_text_Warrawunda.csv',header=None)
warra

,0
0,West Coast Meat Solutions PLU 10316T BBQ Sausa...
1,Beef Mince 80cl Frozen (KG) Packed On Best Bef...
2,26 Boneless Beef Mince 90VL 5mm 3 x 5kg Pieces...
3,Chilled Keep Refrigerated - If Frozen Keep Frozen
4,"Avon Valley Beef 503 Northam Pithara Road, Pit..."
5,Product of Australia Boneless Beef 01199036538...
6,Keep Frozen Product of Australia 0030000000001...
7,West Coast Meat Solutions PLU 10718T VAC Piece...
8,47 Boneless Beef Yearling Diced Large 3x5kg Fr...
9,Chilled Keep Refrigerated - If Frozen Keep Frozen


In [3]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
modelname = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForQuestionAnswering.from_pretrained(modelname)

C:\Users\Loc Sensei\anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from datetime import datetime, timedelta
import re
questions = [
    "What is the name of the food ?",
    "What is the total weight ?",
    "What is the expiry date ?"]

# Function to compute the best before date from packed date
def compute_best_before_date(packed_date, months):
    try:
        packed_date_obj = datetime.strptime(packed_date, "%d/%m/%y")  # Handle two-digit year
        best_before_date_obj = packed_date_obj + timedelta(days=months*30)  # Approximate month length as 30 days
        return best_before_date_obj.strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error computing best before date: {e}")
        return None

# Function to parse dates in DDMMYY format
def parse_ddmmyy(date_str):
    try:
        return datetime.strptime(date_str, "%d%m%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None

# Function to parse best before date in YYMMDD format
def parse_yymmdd(date_str):
    try:
        return datetime.strptime(date_str, "%d%m%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None

# Function to parse dates in DD.MMM.YYYY format
def parse_ddmmmyyyy(date_str):
    try:
        return datetime.strptime(date_str, "%d.%b.%Y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD.MM.YYYY format
def parse_ddmmyyyy(date_str):
    try:
        return datetime.strptime(date_str, "%d.%m.%Y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD MMM YY format
def parse_ddmmmyy(date_str):
    try:
        return datetime.strptime(date_str, "%d %b %y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    

# Function to parse dates in DD-MMM-YYYY format
def parse_ddmmmyyyy_hyphen(date_str):
    try:
        return datetime.strptime(date_str, "%d-%b-%Y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD-MMM-YY format
def parse_ddmmmyy_hyphen(date_str):
    try:
        return datetime.strptime(date_str, "%d-%b-%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD-MM-YY format
def parse_ddmmyy_hyphen(date_str):
    try:
        return datetime.strptime(date_str, "%d-%m-%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD/MM/YY format
def parse_ddmmyy_slash(date_str):
    try:
        return datetime.strptime(date_str, "%d/%m/%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None


In [5]:
# Function to get answer for a single question
def get_answer(question, context):
    
    dates = []
    # Prepare the inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs["input_ids"].tolist()[0]

    # Get model outputs
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer span
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Decode the answer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    
    if "total weight" in question.lower():
        # First, look for multiple quantities and weights
        weights = re.findall(r'(\d+)\s*x\s*(\d+\.?\d*)\s*kg', context, re.IGNORECASE)
        total_weight = 0
        for quantity, weight in weights:
            total_weight += int(quantity) * float(weight)
        if total_weight > 0:
            return f"{total_weight:.2f} kg"
        else:
            # If no multiple quantities are found, look for single weight entries
            match = re.search(r'(Net Weight|Net Wt|Net Ungt):?\s*(\d+\.?\d*)', context, re.IGNORECASE)
            if match:
                return f"{match.group(2)} kg"
            else:
                match = re.search(r'(\d+\.?\d*)\s*kg', context, re.IGNORECASE)
                if match:
                    return f"{match.group(1)} kg"
                else:
                    return "Total weight information not found."
    
        
    elif any(term in question.lower() for term in ["expiry date", "use by date", "best before date", "expiry"]):
        # Check for standard date format
        date_match_standard = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2}/\d{2}/\d{4})', context, re.IGNORECASE)
        if date_match_standard:
            dates.extend([datetime.strptime(match[1], "%d/%m/%Y") for match in date_match_standard])

        # Check for long date format
        date_match_long = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{1,2} \w+ \d{4})', context, re.IGNORECASE)
        if date_match_long:
            dates.extend([datetime.strptime(match[1], "%d %b %Y") for match in date_match_long])

        # Check if production date has "date" word before and best before random int months
        packed_date_match = re.search(r'Date (\d{1,2}/\d{1,2}/\d{2,4})', context, re.IGNORECASE)
        best_before_match = re.search(r'Best Before (\d+) Months from Packed Date', context, re.IGNORECASE)
        if packed_date_match and best_before_match:
            packed_date = packed_date_match.group(1)
            months = int(best_before_match.group(1))
            best_before_date = compute_best_before_date(packed_date, months)
            dates.append(datetime.strptime(best_before_date, "%d/%m/%Y"))

        # Check if production date has no "date" word before and best before random int months
        date_match_unlabeled = re.search(r'\b(\d{1,2}/\d{1,2}/\d{2,4})\b', context)
        best_before_match = re.search(r'Best Before (\d+) Months from Packed Date', context, re.IGNORECASE)
        if date_match_unlabeled and best_before_match:
            packed_date = date_match_unlabeled.group(1)
            months = int(best_before_match.group(1))
            best_before_date = compute_best_before_date(packed_date, months)
            dates.append(datetime.strptime(best_before_date, "%d/%m/%Y"))

        # Check for DDMMYY format
        date_match_ddmmyy = re.findall(r'Best Before:\s*(\d{6})', context, re.IGNORECASE)
        if date_match_ddmmyy:
            dates.extend([datetime.strptime(parse_ddmmyy(match), "%d/%m/%Y") for match in date_match_ddmmyy])

        # Check for YYMMDD format
        date_match_yymmdd = re.findall(r'Best Before:\s*(\d{6})', context, re.IGNORECASE)
        if date_match_yymmdd:
            dates.extend([datetime.strptime(parse_yymmdd(match), "%d/%m/%Y") for match in date_match_yymmdd])

        # Check for DD.MMM.YYYY format
        date_match_ddmmmyyyy = re.findall(r'Best Before\s*(\d{2}\.\w{3}\.\d{4})', context, re.IGNORECASE)
        if date_match_ddmmmyyyy:
            dates.extend([datetime.strptime(parse_ddmmmyyyy(match), "%d/%m/%Y") for match in date_match_ddmmmyyyy])

        # Check for DD.MM.YYYY format
        date_match_ddmmyyyy = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2}\.\d{2}\.\d{4})', context, re.IGNORECASE)
        if date_match_ddmmyyyy:
            dates.extend([datetime.strptime(parse_ddmmyyyy(match[1]), "%d/%m/%Y") for match in date_match_ddmmyyyy])

        # Check for DD MMM YY format
        date_match_ddmmmyy = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2} \w{3} \d{2})', context, re.IGNORECASE)
        if date_match_ddmmmyy:
            dates.extend([datetime.strptime(parse_ddmmmyy(match[1]), "%d/%m/%Y") for match in date_match_ddmmmyy])

        # Check for DD-MMM-YYYY format
        date_match_ddmmmyyyy_hyphen = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2}-\w{3}-\d{4})', context, re.IGNORECASE)
        if date_match_ddmmmyyyy_hyphen:
            dates.extend([datetime.strptime(parse_ddmmmyyyy_hyphen(match[1]), "%d/%m/%Y") for match in date_match_ddmmmyyyy_hyphen])

        # Check for DD-MMM-YY format
        date_match_ddmmmyy_hyphen = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2}-\w{3}-\d{2})', context, re.IGNORECASE)
        if date_match_ddmmmyy_hyphen:
            dates.extend([datetime.strptime(parse_ddmmmyy_hyphen(match[1]), "%d/%m/%Y") for match in date_match_ddmmmyy_hyphen])
            
        # Check for DD-MM-YY format
        date_match_ddmmyy_hyphen = re.findall(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2}-\d{2}-\d{2})', context, re.IGNORECASE)
        if date_match_ddmmyy_hyphen:
            dates.extend([datetime.strptime(parse_ddmmyy_hyphen(match[1]), "%d/%m/%Y") for match in date_match_ddmmyy_hyphen])
            
            
        # Check for latest DD-MMM-YY after another DD-MMM-YY
        date_match_ddmmmyy_hyphen = re.findall(r'Best Before\s+(\d{2}-\w{3}-\d{2})\s+(\d{2}-\w{3}-\d{2})', context, re.IGNORECASE)
        if date_match_ddmmmyy_hyphen:
            dates.extend([datetime.strptime(parse_ddmmmyy_hyphen(match[1]), "%d/%m/%Y") for match in date_match_ddmmmyy_hyphen])
            
        # Check for latest DD/MM/YY after another DD/MM/YY
        date_match_ddmmyy_slash = re.findall(r'Best Before\s+\d+\s+(\d{2}/\d{2}/\d{2,4})\s+(\d{2}/\d{2}/\d{2,4})', context, re.IGNORECASE)
        if date_match_ddmmyy_slash:
            dates.extend([datetime.strptime(parse_ddmmyy_slash(match[1]), "%d/%m/%Y") for match in date_match_ddmmyy_slash])
            
            
        # Check for unlabeled date formats
        date_match_unlabeled = re.findall(r'\b(\d{2}/\d{2}/\d{4})\b', context)
        if date_match_unlabeled:
            dates.extend([datetime.strptime(match, "%d/%m/%Y") for match in date_match_unlabeled])


        if dates:
            latest_date = max(dates)
            return latest_date.strftime("%d/%m/%Y")
        
        return "There's no expiry date."
    
    elif "food" in question.lower() or "product" in question.lower():
        # Extract the product name or food description
        match = re.search(r'Packed Date \d{1,2}/\d{1,2}/\d{2,4} (.+?) (?:\d+\s*KG|Keep Frozen|Product of Australia)', context, re.IGNORECASE)
        if match:
            product_description = match.group(1).strip()
            # Remove any trailing "Product of Australia" if present
            product_description = re.sub(r'\bProduct of Australia\b', '', product_description, flags=re.IGNORECASE).strip()
            return product_description

    return answer.strip()

In [6]:
context = adrian[0][27]

for question in questions:
    answer = get_answer(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What is the name of the food ?
Answer: top cut foods

Question: What is the total weight ?
Answer: 15.44 kg

Question: What is the expiry date ?
Answer: 23/07/2024



In [7]:
food = []
expiry = []
weight = []
question = "What is the name of the food ?"
for context in adrian[0]:
    answer = get_answer(question, context)
    food.append(answer)
      
question1 = "What is the expiry date ?"
for context in adrian[0]:
    answer = get_answer(question1, context)
    expiry.append(answer)
    
question2 = "What is the total weight ?"
for context in adrian[0]:
    answer = get_answer(question2, context)
    weight.append(answer)
    
dictf = {'food': food, 'expiry_date': expiry, 'net_weight': weight} 
   
df = pd.DataFrame(dictf)
   
df


,food,expiry_date,net_weight
0,boneless lamb,09/07/2026,13.28 kg
1,beef burger,14/05/2025,13.63 kg
2,frozen octopus tentacles sous vide,24/06/2026,10.00 kg
3,frozen ocean trout fillet,There's no expiry date.,7.00 kg
4,gourmet beef burger,15/06/2025,7.2 kg
5,tassal tasmanian salmon,09/07/2025,Total weight information not found.
6,packed snapper,26/12/2025,5 kg
7,frozen thigh fillet,12/06/2025,10 kg
8,m & j chickens,05/07/2025,4.8 kg
9,Snapper FLTS Saddletail FRZN,17/12/2025,5 kg


In [8]:
food1 = []
expiry1 = []
weight1 = []
question = "What is the name of the food ?"
for context in warra[0]:
    answer = get_answer(question, context)
    food1.append(answer)
      
question1 = "What is the expiry date ?"
for context in warra[0]:
    answer = get_answer(question1, context)
    expiry1.append(answer)
    
question2 = "What is the total weight ?"
for context in warra[0]:
    answer = get_answer(question2, context)
    weight1.append(answer)
    
dictff = {'food': food1, 'expiry_date': expiry1, 'net_weight': weight1} 
   
df1 = pd.DataFrame(dictff)
   
df1

,food,expiry_date,net_weight
0,west coast meat solutions plu 10316t bbq sausages,28/03/2025,14.98 kg
1,beef mince,27/03/2025,6.00 kg
2,p - 590 frozen of australia,23/01/2025,15.00 kg
3,chilled keep refrigerated,There's no expiry date.,Total weight information not found.
4,,27/01/2025,20.00 kg
5,boneless beef,25/07/2024,12.92 kg
6,keep frozen product of australia,10/07/2025,14.94 kg
7,west coast meat solutions,22/11/2025,15.06 kg
8,p - 590,05/07/2025,15.00 kg
9,chilled keep refrigerated,There's no expiry date.,Total weight information not found.


## Accuracy of model (Adrian folder)

In [9]:
no_weight = "Total weight information not found."
no_expiry = "There's no expiry date."
ground_truths = {
    "food_names": [
    "boneless lamb", "beef burger", "octopus tentacles sous vide", "trout fillet", "beef burger",
    "tassal tasmanian salmon", "packed snapper flts red crimson", "thigh fillet", "m & j chickens",
    "snapper flts saddletail frzn", "barramundi", "barramundi", "mussels", "boneless beef",
    "thin franks vienna mal6", "pastrami", "dani sal", "chorizo", "fleischwurst pariser",
    "swiss meatloaf", "cabanossi", "pastrami", "silverside", "mortadella", "soccerball ham", "bacon",
    "h/pepp", "boneless beef", "boneless beef", "cheese", "crunch"
],
    "net_weights": ["13.28 kg", "13.63 kg","10.00 kg","7.00 kg","7.2 kg",no_weight,"5 kg","10 kg","4.8 kg",
                   "5 kg", "8 kg","8 kg","12.00 kg","15.16 kg","13.78 kg","5.00 kg","2.00 kg","0.390 kg",
                   "0.428 kg","0.492 kg","12.70 kg","5.00 kg","5.00 kg","8.29 kg","13.14 kg",
                   "15.00 kg","4.00 kg","15.44 kg","13.70 kg","5.18 kg","10.00 kg"],
    "expiry_dates": ["09/07/2026","14/05/2025","24/06/2026",no_expiry,"15/06/2025","09/07/2025","26/12/2025",
                    "12/06/2025","05/07/2025","17/12/2025","17/09/2024","27/03/2023","06/04/2025","25/07/2024",
                    "13/08/2024","15/08/2024","26/08/2024",no_expiry,"27/08/2024","03/08/2024","15/08/2024",
                    "15/08/2024","08/08/2024","19/08/2024","02/09/2024","12/08/2024","09/08/2024","23/07/2024",
                    "25/07/2024","30/08/2024","04/10/2024"]
}

predicted_food_names = []
predicted_net_weights = []
predicted_expiry_dates = []

for text in adrian[0]:
    predicted_food_names.append(get_answer(question,text))
    predicted_expiry_dates.append(get_answer(question1,text))
    predicted_net_weights.append(get_answer(question2,text))

# Calculate accuracy for each category
def calculate_accuracy(predictions, ground_truths):
    correct = sum([1 for pred, gt in zip(predictions, ground_truths) if pred == gt])
    total = len(ground_truths)
    return correct / total * 100

food_name_accuracy = calculate_accuracy(predicted_food_names, ground_truths["food_names"])
net_weight_accuracy = calculate_accuracy(predicted_net_weights, ground_truths["net_weights"])
expiry_date_accuracy = calculate_accuracy(predicted_expiry_dates, ground_truths["expiry_dates"])

print(f"Food Name Accuracy: {food_name_accuracy:.2f}%")
print(f"Net Weight Accuracy: {net_weight_accuracy:.2f}%")
print(f"Expiry Date Accuracy: {expiry_date_accuracy:.2f}%")

Food Name Accuracy: 54.84%
Net Weight Accuracy: 96.77%
Expiry Date Accuracy: 96.77%


In [10]:
# Compare ground truth versus predictions
correct = 0
total = len(ground_truths.get("food_names"))

for gt, pred in zip(ground_truths.get("food_names"), predicted_food_names):
    if gt == pred:
        correct += 1
    else:
        print(f"Mismatch: Ground Truth = {gt}, Prediction = {pred}")

# Calculate accuracy
accuracy = (correct / total) * 100

print(f"Total correct: {correct}/{total}")

Mismatch: Ground Truth = octopus tentacles sous vide, Prediction = frozen octopus tentacles sous vide
Mismatch: Ground Truth = trout fillet, Prediction = frozen ocean trout fillet
Mismatch: Ground Truth = beef burger, Prediction = gourmet beef burger
Mismatch: Ground Truth = packed snapper flts red crimson, Prediction = packed snapper
Mismatch: Ground Truth = thigh fillet, Prediction = frozen thigh fillet
Mismatch: Ground Truth = snapper flts saddletail frzn, Prediction = Snapper FLTS Saddletail FRZN
Mismatch: Ground Truth = boneless beef, Prediction = keep chilled product of australia beef s / strp / in fillet pc vac / 15 / 16kg mv / vac net wt 15. 16 kg pkd on 18 jul 2024 best before 25 jul 2024 103983 boneless beef
Mismatch: Ground Truth = thin franks vienna mal6, Prediction = thin franks
Mismatch: Ground Truth = mortadella, Prediction = primo bulk mortadella pan
Mismatch: Ground Truth = soccerball ham, Prediction = primo soccerball ham
Mismatch: Ground Truth = bacon, Prediction = m

In [11]:
# Compare ground truth versus predictions
correct = 0
total = len(ground_truths.get("net_weights"))

for gt, pred in zip(ground_truths.get("net_weights"), predicted_net_weights):
    if gt == pred:
        correct += 1
    else:
        print(f"Mismatch: Ground Truth = {gt}, Prediction = {pred}")

# Calculate accuracy
accuracy = (correct / total) * 100

print(f"Total correct: {correct}/{total}")

Mismatch: Ground Truth = 5.18 kg, Prediction = 2.5 kg
Total correct: 30/31


In [12]:
# Compare ground truth versus predictions
correct = 0
total = len(ground_truths.get("expiry_dates"))

for gt, pred in zip(ground_truths.get("expiry_dates"), predicted_expiry_dates):
    if gt == pred:
        correct += 1
    else:
        print(f"Mismatch: Ground Truth = {gt}, Prediction = {pred}")

# Calculate accuracy
accuracy = (correct / total) * 100

print(f"Total correct: {correct}/{total}")

Mismatch: Ground Truth = 27/03/2023, Prediction = There's no expiry date.
Total correct: 30/31


## Accuracy of model (Warra folder)

In [21]:
no_weight = "Total weight information not found."
no_expiry = "There's no expiry date."
no_food = "There's no food found"
ground_truths = {
    "food_names": [
    "bbq sausages", "beef mince", "boneless beef mince", no_food, "beef",
    "boneless beef", "boneless beef", "boneless beef", "boneless beef",
    no_food, "chicken chubbypieces", "thigh fillet", "chicken mixpcs", "herb & garlic lamb sausages",
    "herb & garlic sausages", "bonless lamb", "boneless pork", "boneless pork", "pork loin chop",
    "pork (mince)", "bone in pork", "boneless pork", "boneless pork", "boneless pork", "mid bacon", "sausage bbq",
    "sausage italian", "bacon", "boneless beef", "yg t-bone sliced yarrie 350"
],
    "net_weights": ["14.98 kg", "6.00 kg","15.30 kg",no_weight,"20.00 kg","12.92 kg","14.94 kg","15.06 kg","14.96 kg",
                   no_weight, "12.00 kg","10 kg","16 kg","14.08 kg","13.54 kg","13.10 kg","12.78 kg","12.72 kg",
                   "10.00 kg","15.11 kg","12.64 kg","15.56 kg","16.10 kg","11.20 kg","15.00 kg",
                   "14.82 kg","15.00 kg","10.00 kg","14.95 kg",no_weight],
    
    "expiry_dates": ["28/03/2025","27/03/2025","23/01/2025",no_expiry,"27/01/2025","25/07/2024","10/07/2025",
                    "22/11/2025","05/07/2025",no_expiry,"10/08/2024","26/06/2025","16/06/2023","14/01/2026",
                    "03/05/2025","20/09/2025","21/05/2025","26/06/2025","19/11/2023","10/12/2024","30/05/2025",
                    "28/03/2025","20/06/2025","24/06/2025","06/08/2024","18/06/2025","15/04/2025","03/07/2024",
                    "30/10/2025",no_expiry]
}

predicted_food_names = []
predicted_net_weights = []
predicted_expiry_dates = []

for text in warra[0]:
    predicted_food_names.append(get_answer(question,text))
    predicted_expiry_dates.append(get_answer(question1,text))
    predicted_net_weights.append(get_answer(question2,text))

# Calculate accuracy for each category
def calculate_accuracy(predictions, ground_truths):
    correct = sum([1 for pred, gt in zip(predictions, ground_truths) if pred == gt])
    total = len(ground_truths)
    return correct / total * 100

food_name_accuracy = calculate_accuracy(predicted_food_names, ground_truths["food_names"])
net_weight_accuracy = calculate_accuracy(predicted_net_weights, ground_truths["net_weights"])
expiry_date_accuracy = calculate_accuracy(predicted_expiry_dates, ground_truths["expiry_dates"])

print(f"Food Name Accuracy: {food_name_accuracy:.2f}%")
print(f"Net Weight Accuracy: {net_weight_accuracy:.2f}%")
print(f"Expiry Date Accuracy: {expiry_date_accuracy:.2f}%")

Food Name Accuracy: 43.33%
Net Weight Accuracy: 93.33%
Expiry Date Accuracy: 100.00%


In [22]:
# Compare ground truth versus predictions
correct = 0
total = len(ground_truths.get("food_names"))

for gt, pred in zip(ground_truths.get("food_names"), predicted_food_names):
    if gt == pred:
        correct += 1
    else:
        print(f"Mismatch: Ground Truth = {gt}, Prediction = {pred}")

# Calculate accuracy
accuracy = (correct / total) * 100

print(f"Total correct: {correct}/{total}")

Mismatch: Ground Truth = bbq sausages, Prediction = west coast meat solutions plu 10316t bbq sausages
Mismatch: Ground Truth = boneless beef mince, Prediction = p - 590 frozen of australia
Mismatch: Ground Truth = There's no food found, Prediction = chilled keep refrigerated
Mismatch: Ground Truth = beef, Prediction = 
Mismatch: Ground Truth = boneless beef, Prediction = keep frozen product of australia
Mismatch: Ground Truth = boneless beef, Prediction = west coast meat solutions
Mismatch: Ground Truth = boneless beef, Prediction = p - 590
Mismatch: Ground Truth = There's no food found, Prediction = chilled keep refrigerated
Mismatch: Ground Truth = thigh fillet, Prediction = frozen thigh fillet
Mismatch: Ground Truth = chicken mixpcs, Prediction = keep frozen
Mismatch: Ground Truth = herb & garlic lamb sausages, Prediction = west coast meat solutions plu 40012 bulk pieces : 3 herb & garlic lamb sausages
Mismatch: Ground Truth = herb & garlic sausages, Prediction = west coast meat sol

In [23]:
# Compare ground truth versus predictions
correct = 0
total = len(ground_truths.get("net_weights"))

for gt, pred in zip(ground_truths.get("net_weights"), predicted_net_weights):
    if gt == pred:
        correct += 1
    else:
        print(f"Mismatch: Ground Truth = {gt}, Prediction = {pred}")

# Calculate accuracy
accuracy = (correct / total) * 100

print(f"Total correct: {correct}/{total}")

Mismatch: Ground Truth = 15.30 kg, Prediction = 15.00 kg
Mismatch: Ground Truth = 14.96 kg, Prediction = 15.00 kg
Total correct: 28/30


In [24]:
# Compare ground truth versus predictions
correct = 0
total = len(ground_truths.get("expiry_dates"))

for gt, pred in zip(ground_truths.get("expiry_dates"), predicted_expiry_dates):
    if gt == pred:
        correct += 1
    else:
        print(f"Mismatch: Ground Truth = {gt}, Prediction = {pred}")

# Calculate accuracy
accuracy = (correct / total) * 100

print(f"Total correct: {correct}/{total}")

Total correct: 30/30
